In [1]:
import os
import glob
from tqdm import tqdm

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import silence_tensorflow.auto
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
import keras
import random
import numpy as np
import matplotlib.pyplot as plt

E0000 00:00:1738428489.235415   40186 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738428489.241865   40186 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
H = 512  # height
W = 512  # width
C = 3  # channels
h = 16  # heads

P = 16  # patch size
assert H == W
assert H % P == 0

D_model: int = 512  # transformer latent dim
D_head: int = 128  # dim of each head
D_fcn: int = 2048  # FCN hidden dim
num_layers: int = 4  # transformer depth
N: int = (H * W) // (P * P)  # Number of patches
BS = 4

# assert h * D_head == D_model

In [3]:
FLOAT = tf.float32

In [4]:
def load_and_validate(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_image(img, channels=C, expand_animations=False)
    img = tf.divide(tf.cast(img, dtype=FLOAT), 255.0)
    is_valid = tf.reduce_all(tf.equal(tf.shape(img), tf.constant((H, W, C))))

    return img, is_valid


dataset_path = "/mnt/Data/ML/datasets/portraits"
num_samples = 5000


all_files = [
    os.path.join(dataset_path, f)
    for f in os.listdir(dataset_path)
    if f.endswith((".jpg", ".png"))
]
random.shuffle(all_files)
selected_files = all_files[:num_samples]
dataset = tf.data.Dataset.from_tensor_slices(selected_files)
dataset = dataset.map(load_and_validate, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.filter(lambda img, is_valid: is_valid)  # Keep valid images
dataset = dataset.map(lambda img, is_valid: img)  # remove unused feature
dataset = dataset.map(lambda img: tf.ensure_shape(img, (H, W, C)))
print(f"Total files: {len(selected_files)}")
valid_count = dataset.reduce(tf.constant(0), lambda x, _: x + 1).numpy()
print(f"Valid images count: {valid_count}")
assert valid_count, "Everything's gone"

I0000 00:00:1738428496.132556   40186 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2181 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Total files: 5000
Valid images count: 5000


In [6]:
def viz_img(img):
    img = tf.cast(img, tf.float32)
    plt.imshow(tf.squeeze(img).numpy(), cmap="gray")
    plt.colorbar()
    plt.show()


def viz_mask(mask):
    plt.imshow(tf.squeeze(mask).numpy(), cmap="gray", vmin=0, vmax=1)
    plt.colorbar()
    plt.show()

def random_visibility_mask():
    x1 = tf.random.uniform(shape=(), minval=0, maxval=W - 100, dtype=tf.int32)
    y1 = tf.random.uniform(shape=(), minval=0, maxval=H - 100, dtype=tf.int32)
    x2 = tf.random.uniform(shape=(), minval=x1 + 100, maxval=W + 1, dtype=tf.int32)
    y2 = tf.random.uniform(shape=(), minval=y1 + 100, maxval=H + 1, dtype=tf.int32)
    # tf.print(x1,x2,y1,y2)

    mask = tf.ones((H, W), dtype=tf.bool)
    mask = tf.tensor_scatter_nd_update(
        mask,
        indices=tf.stack(
            [
                tf.repeat(tf.range(y1, y2), x2 - x1),
                tf.tile(tf.range(x1, x2), [y2 - y1]),
            ],
            axis=-1,
        ),
        updates=tf.zeros([(y2 - y1) * (x2 - x1)], dtype=tf.bool),
    )
    return tf.expand_dims(mask, -1)  # expand channel wise


def mask_area(mask):
    return tf.reduce_sum(tf.cast(mask, tf.int32))


def extract_patches(image: tf.Tensor) -> tf.Tensor:
    "R^{BS x H x W x C} -> R^{BS x N x P^2 x C}"
    # print(image.dtype)

    patches: tf.Tensor = tf.image.extract_patches(
        images=image,  # Add batch dim
        sizes=[1, P, P, 1],  # Patch size
        strides=[1, P, P, 1],  # Step size
        rates=[1, 1, 1, 1],  # No dilation
        padding="VALID",
    )
    BS, H_prime, W_prime, _ = tf.unstack(tf.shape(patches))

    # Reshape patches to [BS, H' * W', P*P, C]
    patches = tf.reshape(patches, [BS, H_prime * W_prime, P * P, -1])

    return patches


def patches_to_imgs(patches: tf.Tensor) -> tf.Tensor:
    "R^{BS x N x P^2 x C} -> R^{BS x H x W x C}"
    BS = tf.shape(patches)[0]
    grid_size = H // P  # same as W // P

    patches = tf.reshape(patches, [BS, grid_size, grid_size, P, P, C])
    patches = tf.transpose(patches, perm=[0, 1, 3, 2, 4, 5])

    image = tf.reshape(patches, [BS, grid_size * P, grid_size * P, C])

    return image


def process_mask(mask: tf.Tensor):
    "R^{BS x H x W} -> tuple[R^{BS x N}, R^{BS x N x N}]"
    # viz_mask(mask)
    BS = tf.shape(mask)[0]
    mask_pooled = tf.nn.max_pool2d(
        tf.cast(
            tf.logical_not(mask), dtype=tf.int8
        ),  # insane shit happaned here with mask_inverted
        ksize=[P, P],
        strides=[P, P],
        padding="VALID",
    )
    mask_pooled = tf.logical_not(tf.cast(mask_pooled, tf.bool))
    # viz_mask(mask_pooled)
    mask_pooled = tf.reshape(mask_pooled, [BS, N])
    mask_expanded = tf.expand_dims(mask_pooled, axis=1)  # (BS, 1, N)
    mask_expanded = tf.tile(mask_expanded, [1, N, 1])  # (BS, N, N)
    A = tf.where(
        mask_expanded,
        tf.constant(0.0, dtype=FLOAT),  # zero penanly
        tf.constant(-float("inf"), dtype=FLOAT),  # inf penalty
    )
    # tf.print(tf.shape(A))

    return mask_pooled, A

In [7]:
class PatchEmbedding(keras.layers.Layer):
    def __init__(self):
        super().__init__(dtype=FLOAT)  # wtf
        self.proj = keras.layers.Dense(D_model, dtype=FLOAT)  # (P² * C) -> D_model
        self.positional_embedding = self.add_weight(
            shape=(1, N, D_model),
            initializer="random_normal",
            trainable=True,
            name="positional_embedding",
            dtype=FLOAT,
        )

    # def build(self, input_shape):
    #     self.call(tf.zeros((BS, N , P**2 * C)))
    #     self.built = True

    def call(self, patches_flat: tf.Tensor):
        # R^{N x (P^2 . C)} -> R^{N x D_model}
        assert patches_flat.dtype == FLOAT
        X = self.proj(patches_flat)  # (N, D_model)
        assert X.dtype == FLOAT
        X += self.positional_embedding  # (N, D_model)
        assert X.dtype == FLOAT
        return X


class MultiHeadAttention(keras.layers.Layer):
    def __init__(self):
        super().__init__(dtype=FLOAT)
        # Project to h * D_head dimensions
        self.W_Q = keras.layers.Dense(h * D_head, dtype=FLOAT)
        self.W_K = keras.layers.Dense(h * D_head, dtype=FLOAT)
        self.W_V = keras.layers.Dense(h * D_head, dtype=FLOAT)
        # Project back to D_model
        self.W_O = keras.layers.Dense(D_model, dtype=FLOAT)

    def call(self, X, A):
        # X: R^{BS x N x D_model}
        # A: R^{BS x N x N}
        # returns: R^{BS x N x D_model}

        # In the standard implementation, each head has its own separate projection matrices. However, a common optimization is to project the input into h * D_head dimensions (which is D_model) with a single large projection, then split into h heads. So, if D_model = h * D_head, then using a Dense(D_model) for Q, K, V and then splitting into h heads each of D_head is equivalent to having h separate projections. This is a standard approach because it's more efficient to compute all heads in parallel with a single matrix multiplication rather than h separate ones.
        # So the optimal way is to use combined projections.
        Q = self.W_Q(X)  # (BS, N, h * D_head)
        K = self.W_K(X)  # (BS, N, h * D_head)
        V = self.W_V(X)  # (BS, N, h * D_head)
        

        Q = tf.reshape(Q, (-1, N, h, D_head))  # (BS, N, h, D_head)
        K = tf.reshape(K, (-1, N, h, D_head))
        V = tf.reshape(V, (-1, N, h, D_head))

        # Transpose for attention computation
        Q = tf.transpose(Q, [0, 2, 1, 3])  # (BS, h, N, D_head)
        K = tf.transpose(K, [0, 2, 1, 3])
        V = tf.transpose(V, [0, 2, 1, 3])
        # scaled dot-product attention
        attn_scores = tf.matmul(Q, K, transpose_b=True)  # (BS, h, N, N)
        attn_scores /= tf.math.sqrt(
            tf.cast(D_head, attn_scores.dtype)
        )  # scale by sqrt(D_head)

        A = tf.expand_dims(A, 1)  # (BS, 1, N, N)
        attn_scores += A  # Broadcast to all heads

        attn_weights = tf.nn.softmax(attn_scores, axis=-1)  # (BS, h, N, N)

        output = tf.matmul(attn_weights, V)  # (BS, h, N, D_head)
        output = tf.transpose(output, [0, 2, 1, 3])  # (BS, N, h, D_head)
        output = tf.reshape(output, (-1, N, h * D_head))  # (BS, N, h * D_head)
        output = self.W_O(output)  # (BS, N, D_model)
        return output


class TransformerBlock(keras.layers.Layer):
    def __init__(self):
        super().__init__(dtype=FLOAT)
        self.attn = MultiHeadAttention()
        self.norm1 = keras.layers.LayerNormalization(dtype=FLOAT)
        self.norm2 = keras.layers.LayerNormalization(dtype=FLOAT)
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(D_fcn, activation="gelu", dtype=FLOAT),
                keras.layers.Dense(D_model, dtype=FLOAT),
                keras.layers.Dropout(0.1, dtype=FLOAT),
            ],
        )

    def call(self, X, A):
        "R^{N x D_model} -> R^{N x D_model}"
        X = self.norm1(X + self.attn(X, A))
        X = self.norm2(X + self.ffn(X))
        return X


class Decoder(keras.layers.Layer):
    def __init__(self):
        super().__init__(dtype=FLOAT)
        self.proj = keras.layers.Dense(P * P * C, dtype=FLOAT)

    def call(self, X):
        "R^{BS x N x D_model} -> R^{BS x N x P x P x C}"
        BS = tf.shape(X)[0]
        X = self.proj(X)
        X = tf.reshape(X, (BS, N, P, P, C))
        return X


class ImageInpaintingTransformer(keras.Model):
    def __init__(self):
        super().__init__(dtype=FLOAT)
        self.embed = PatchEmbedding()
        self.transformer_blocks = [TransformerBlock() for _ in range(num_layers)]
        self.decoder = Decoder()

    def build(self, input_shape):
        BS = input_shape[0]
        dummy_images = tf.zeros((BS, H, W, C), dtype=FLOAT)  # THIS WASTED 40 MINUTES
        dummy_masks = tf.stack([random_visibility_mask() for _ in range(BS)])
        self.call(dummy_images, dummy_masks)
        self.built = True

    def call(self, image, mask):
        patches = extract_patches(image)
        mask_pooled, A = process_mask(mask)
        BS = tf.shape(patches)[0]
        patches_flat = tf.reshape(patches, [BS, N, P**2 * C])
        # tf.print(tf.shape(patches_flat))
        X = self.embed(patches_flat)
        for block in self.transformer_blocks:
            X = block(X, A)
        reconstructed_patches = self.decoder(X)  # R^{BS x N x P x P x C}
        return patches_to_imgs(reconstructed_patches)

model = ImageInpaintingTransformer()
model.build((BS, H, W, C))
model.summary()

Model: "image_inpainting_transformer_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ patch_embedding_1               │ ?                      │       918,016 │
│ (PatchEmbedding)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_4             │ ?                      │     6,302,720 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_5             │ ?                      │     6,302,720 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_6             │ ?                      │     6,302,720 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_7             │ ?                      │     6,302,720 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_1 (Decoder)             │ ?                      │       393,984 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,522,880 (101.18 MB)

 Trainable params: 26,522,880 (101.18 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# # Tests patching and depatching

# img = next(iter(dataset.take(1)))

# viz_img(img)
# patched = extract_patches(tf.expand_dims(img ,0))
# tf.print(tf.shape(patched))

# recreated_img = patches_to_imgs(patched)
# viz_img(recreated_img)

In [8]:
def prepare_sample(image):
    mask = random_visibility_mask()
    return image, mask


ds_masks = dataset.map(prepare_sample)
train_count = int(valid_count * 0.8)
test_count = int(valid_count * 0.1)
val_count = valid_count - train_count - test_count

train_ds = ds_masks.take(train_count).batch(BS)
test_ds = ds_masks.skip(train_count).take(test_count).batch(BS)
val_ds = ds_masks.skip(train_count + test_count).take(val_count).batch(BS)

In [9]:
@tf.function
def l1_masked_loss(y_true: tf.Tensor, y_pred: tf.Tensor, obsvmask: tf.Tensor):
    inpaintmask = tf.logical_not(obsvmask)
    whole_image_diff = tf.subtract(y_true, y_pred)
    area = mask_area(inpaintmask)
    masked_diff = tf.multiply(whole_image_diff, tf.cast(inpaintmask, FLOAT))
    return tf.reduce_mean(tf.abs(masked_diff))


optimizer = keras.optimizers.Adam(learning_rate=1e-3)


@tf.function
def train_step(image: tf.Tensor, mask: tf.Tensor):
    with tf.GradientTape() as tape:
        reconstructed_img = model(image, mask)  # N x P x P x C
        loss = l1_masked_loss(image, reconstructed_img, mask)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


@tf.function
def val_step(image: tf.Tensor, mask: tf.Tensor):
    reconstructed_img = model(image, mask)
    loss = l1_masked_loss(image, reconstructed_img, mask)
    return loss


epochs = 10
print("Starting training")
for epoch in range(epochs):
    epoch_loss = 0.0
    steps = 0
    pbar = tqdm(train_ds, desc=f"Epoch {epoch+1}", unit="batch", total=train_count // BS)
    for image_batch, mask_batch in pbar:
        loss = train_step(image_batch, mask_batch)
        epoch_loss += loss
        steps += 1
        # Dynamically update the tqdm bar without spamming stdout
        pbar.set_postfix(loss=f"{loss:.4f}")
    train_loss = epoch_loss / steps

    val_loss_total = 0.0
    val_steps = 0
    pbar_val = tqdm(val_ds, desc=f"Epoch {epoch+1} Validation", unit="batch", total=len(val_ds))
    for val_image_batch, val_mask_batch in pbar_val:
        loss = val_step(val_image_batch, val_mask_batch)
        val_loss_total += loss
        val_steps += 1
        pbar_val.set_postfix(loss=f"{loss:.4f}")
    avg_val_loss = val_loss_total / val_steps

    print(f"Epoch {epoch+1} Summary: Train Loss = {train_loss:.4f} | Validation Loss = {avg_val_loss:.4f}")


Starting training


Epoch 1:   0%|          | 0/1000 [00:21<?, ?batch/s]


ResourceExhaustedError: Graph execution error:

Detected at node image_inpainting_transformer_1_1/transformer_block_7_1/multi_head_attention_7_1/Softmax defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/navid/.pyenv/versions/3.12.8/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/navid/.pyenv/versions/3.12.8/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/navid/.pyenv/versions/3.12.8/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/navid/.pyenv/versions/3.12.8/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/navid/.pyenv/versions/3.12.8/lib/python3.12/asyncio/base_events.py", line 640, in run_forever

  File "/home/navid/.pyenv/versions/3.12.8/lib/python3.12/asyncio/base_events.py", line 1992, in _run

In [11]:
def apply_obsv_mask(image: tf.Tensor, obvmask: tf.Tensor) -> tf.Tensor:
    return tf.multiply(image, tf.cast(obvmask, FLOAT))


def viz_grid(batch: tf.Tensor):
    batch_size: int = batch.shape[0]  # type: ignore
    fig, axes = plt.subplots(nrows=1, ncols=batch_size, figsize=(5, 5))
    for i in range(batch_size):
        # Original image
        axes[i].imshow(tf.cast(batch[i], dtype=tf.float32).numpy())  # type: ignore
        axes[i].axis("off")
    plt.tight_layout()
    plt.show()


def reconstruct(original: tf.Tensor, reconstruct: tf.Tensor, obvmask: tf.Tensor):
    return tf.add(
        tf.multiply(tf.cast(obvmask, FLOAT), original),
        tf.multiply(tf.cast(tf.logical_not(obvmask), FLOAT), reconstruct),
    )

In [ ]:
# Qualitative Eval
# visualize_unbatched_dataset(test_ds, 5)

img, obvmask = next(iter(test_ds.take(1)))
# viz_grid(img)
viz_grid(apply_obsv_mask(img, obvmask))
reconstructed = reconstruct(img, model(img, obvmask), obvmask)
viz_grid(reconstructed)